In [ ]:
import pandas as pd
import os
import math
import time
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import KFold



import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from torchvision import datasets, transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
test_data = pd.read_csv("/kaggle/input/heartbeat/mitbih_test.csv", header = None)
train_data = pd.read_csv("/kaggle/input/heartbeat/mitbih_train.csv", header = None)

X_test, y_test = test_data.iloc[:, :187].values, test_data.iloc[:, 187].values
X_train, y_train = train_data.iloc[:, :187].values, train_data.iloc[:, 187].values

# Assuming X_train, y_train, X_test, y_test are your datasets
# Reshape your data to [batch_size, channels, height, width]
X_train_reshaped = X_train.reshape(-1, 1, 1, 187)
X_test_reshaped = X_test.reshape(-1, 1, 1, 187)

# Convert your datasets to PyTorch tensors
X_train_tensor = torch.tensor(X_train_reshaped, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_reshaped, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# Get the full dataset
merged_X = torch.cat((X_train_tensor, X_test_tensor))
merged_y = torch.cat((y_train_tensor, y_test_tensor))
dataset = TensorDataset(merged_X, merged_y)

# Calculate the number of samples for training and validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

# Split the dataset into training and validation sets
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders for the training, validation, and test sets
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Print the sizes of the datasets
print('Number of training samples:', len(train_subset))
print('Number of validation samples:', len(val_subset))
print('Number of test samples:', len(test_dataset))

In [ ]:
__all__ = ['ResNet', 'resnet34']

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, activation_fn, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.activation_fn = activation_fn()
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation_fn(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.activation_fn(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, activation_fn, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.activation_fn = activation_fn()
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation_fn(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.activation_fn(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.activation_fn(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block, layers, activation_fn, num_classes=5):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.activation_fn = activation_fn()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], activation_fn)
        self.layer2 = self._make_layer(block, 128, layers[1], activation_fn, stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], activation_fn, stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], activation_fn, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Changed to AdaptiveAvgPool2d
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, activation_fn, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, activation_fn, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, activation_fn))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.activation_fn(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet34(activation_fn=nn.ReLU, pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        activation_fn (nn.Module): Activation function to use
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], activation_fn, **kwargs)
    return model


class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(torch.nn.functional.softplus(x))
    
class DSReLU(nn.Module):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    def __init__(self, k=5):
        super(DSReLU, self).__init__()
        self.a = torch.tensor(math.tan(math.radians(75)), dtype=torch.float32, device=device)
        self.b = torch.tensor(math.tan(math.radians(30)), dtype=torch.float32, device=device)
        self.k = k
        self.t = 0

    def forward(self, x):
        t_tensor = torch.tensor(self.t, dtype=x.dtype, device=x.device)
        s_t = self.a + (self.b - self.a) / (1 + torch.exp(-self.k * (t_tensor - 0.5)))
        return torch.where(x > 0, x * s_t, x)

    def increment_t(self, num_batches):
        self.t += 1 / num_batches


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, activation_fn_name, device, patience=5):
    # Move model to the specified device
    model.to(device)
    
    # Create directories for storing artifacts
    artifacts_dir = os.path.join('artifacts', activation_fn_name)
    os.makedirs(artifacts_dir, exist_ok=True)
    
    log_file = os.path.join(artifacts_dir, f'{activation_fn_name}.log')
    best_weights_file = os.path.join(artifacts_dir, 'best_weights.pth')
    top_metrics_file = os.path.join(artifacts_dir, 'top_metrics.csv')
    
    best_loss = float('inf')
    patience_counter = 0
    top_metrics = []
    
    with open(log_file, 'w') as log:
        log.write('Epoch,Train Loss,Val Loss,Train Acc,Val Acc,Train F1,Val F1,Train AUC,Val AUC,Epoch Time\n')
        
        for epoch in range(num_epochs):
            start_time = time.time()
            
             # Increment the t parameter for DSReLU at the start of each epoch
            if isinstance(model.activation_fn, DSReLU):
                model.activation_fn.increment_t(len(train_loader))
                # Calculate and print the slope s(t)
                t_tensor = torch.tensor(model.activation_fn.t, dtype=torch.float32, device=device)
                s_t = model.activation_fn.a + (model.activation_fn.b - model.activation_fn.a) / (1 + torch.exp(-model.activation_fn.k * (t_tensor - 0.5)))
                print(f'Epoch {epoch+1}/{num_epochs}, DSReLU slope: {s_t.item():.4f}')
            
            # Training phase
            model.train()
            train_losses = []
            all_labels = []
            all_preds = []
            all_probs = []
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                train_losses.append(loss.item())
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                all_probs.extend(outputs.softmax(dim=1).detach().cpu().numpy())
            
            train_loss = np.mean(train_losses)
            train_acc = accuracy_score(all_labels, all_preds)
            train_f1 = f1_score(all_labels, all_preds, average='weighted')
            train_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
            
            # Validation phase
            model.eval()
            val_losses = []
            all_labels = []
            all_preds = []
            all_probs = []
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    val_losses.append(loss.item())
                    all_labels.extend(labels.cpu().numpy())
                    all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                    all_probs.extend(outputs.softmax(dim=1).cpu().numpy())
            
            val_loss = np.mean(val_losses)
            val_acc = accuracy_score(all_labels, all_preds)
            val_f1 = f1_score(all_labels, all_preds, average='weighted')
            val_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
            
            epoch_time = time.time() - start_time
            
            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}, Train AUC: {train_auc:.4f}, Val AUC: {val_auc:.4f}, Epoch Time: {epoch_time:.2f}s')
            log.write(f'{epoch+1},{train_loss},{val_loss},{train_acc},{val_acc},{train_f1},{val_f1},{train_auc},{val_auc},{epoch_time}\n')
            
            # Check for best validation loss
            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
                torch.save(model.state_dict(), best_weights_file)
            else:
                patience_counter += 1
            
            # Save top metrics
            top_metrics.append((train_acc, val_acc, train_f1, val_f1, train_auc, val_auc))
            top_metrics.sort(key=lambda x: max(x), reverse=True)
            top_metrics = top_metrics[:5]
            
            if patience_counter >= patience:
                print(f'Early stopping at epoch {epoch+1}')
                break
    
    # Save top metrics to CSV
    df = pd.DataFrame(top_metrics, columns=['Train Acc', 'Val Acc', 'Train F1', 'Val F1', 'Train AUC', 'Val AUC'])
    df.to_csv(top_metrics_file, index=False)
    
    return model

In [ ]:
def cross_validate_model(model_class, dataset, criterion, optimizer_class, num_epochs, activation_fn_name, device, batch_size=32, patience=5, num_folds=5):
    kfold = KFold(n_splits=num_folds, shuffle=True)
    
    fold_metrics = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f'Fold {fold+1}/{num_folds}')
        
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        
        model = model_class()  # Instantiate the model
        optimizer = optimizer_class(model.parameters())  # Instantiate the optimizer
        
        trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, activation_fn_name, device, patience)
        
        # Load best weights
        best_weights_file = os.path.join('artifacts', activation_fn_name, 'best_weights.pth')
        trained_model.load_state_dict(torch.load(best_weights_file))
        
        # Evaluate the model on the validation set
        trained_model.eval()
        all_labels = []
        all_preds = []
        all_probs = []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = trained_model(images)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(outputs.argmax(dim=1).cpu().numpy())
                all_probs.extend(outputs.softmax(dim=1).cpu().numpy())
        
        val_acc = accuracy_score(all_labels, all_preds)
        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        val_auc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
        
        fold_metrics.append((val_acc, val_f1, val_auc))
    
    # Save fold metrics to CSV
    fold_metrics_df = pd.DataFrame(fold_metrics, columns=['Val Acc', 'Val F1', 'Val AUC'])
    fold_metrics_file = os.path.join('artifacts', activation_fn_name, 'cross_val_metrics.csv')
    fold_metrics_df.to_csv(fold_metrics_file, index=False)
    
    return fold_metrics_df

In [ ]:
def experiment(activation_fn, activation_fn_name, num_epochs, patience, weight_decay=False):
    print(f"*********** Training with {activation_fn_name} *******************", end="\n\n")
    resnet_model = resnet34(activation_fn=activation_fn, num_classes=5)
    criterion = nn.CrossEntropyLoss()
    
    if weight_decay:
        optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.01, weight_decay=1e-5)
    else:
        optimizer = torch.optim.Adam(resnet_model.parameters(), lr=0.001)
    
    model = train_model(resnet_model, train_loader, val_loader, criterion, optimizer, num_epochs=num_epochs, 
                        activation_fn_name=activation_fn_name, device=device, patience=patience)
    return model 

In [ ]:
# relu_model = experiment(torch.nn.ReLU, "RELU", num_epochs = 150, patience = 15)
# tanh_model = experiment(torch.nn.Tanh, "Tanh", num_epochs = 150, patience = 15)
# sigmoid_model = experiment(torch.nn.Sigmoid, "Sigmoid", num_epochs = 150, patience = 15)
# mish_model = experiment(Mish, "MISH", num_epochs = 50, patience = 15)
# dsrelu_model = experiment(DSReLU, "DSReLU", num_epochs = 150, patience = 15)
# leakyrelu_model = experiment(torch.nn.LeakyReLU, "LeakyReLU", num_epochs = 150, patience = 15)

In [ ]:
def setup_model(activation_fn):
    model = resnet34(num_classes=5)  
    for name, module in model.named_modules():
        if isinstance(module, nn.ReLU):
            setattr(model, name, activation_fn())
    return model

criterion = nn.CrossEntropyLoss()

def optimizer_fn(params):
    return optim.Adam(params, lr=0.001)  

# Call cross-validation for DSReLU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fold_metrics_df = cross_validate_model(
    lambda: setup_model(DSReLU), 
    dataset, 
    criterion, 
    optimizer_fn, 
    num_epochs=150, 
    activation_fn_name='DSReLU', 
    device=device
)

print(fold_metrics_df)

fold_metrics_df_relu = cross_validate_model(
    lambda: setup_model(nn.ReLU), 
    dataset, 
    criterion, 
    optimizer_fn, 
    num_epochs=150, 
    activation_fn_name='ReLU', 
    device=device
)

print(fold_metrics_df_relu)

fold_metrics_df_tanh = cross_validate_model(
    lambda: setup_model(nn.Tanh), 
    dataset, 
    criterion, 
    optimizer_fn, 
    num_epochs=150, 
    activation_fn_name='Tanh', 
    device=device
)

print(fold_metrics_df_tanh)

fold_metrics_df_sigmoid = cross_validate_model(
    lambda: setup_model(nn.Sigmoid), 
    dataset, 
    criterion, 
    optimizer_fn, 
    num_epochs=150, 
    activation_fn_name='Sigmoid', 
    device=device
)

print(fold_metrics_df_sigmoid)

fold_metrics_df_mish = cross_validate_model(
    lambda: setup_model(Mish), 
    dataset, 
    criterion, 
    optimizer_fn, 
    num_epochs=50, 
    activation_fn_name='Mish', 
    device=device
)

print(fold_metrics_df_mish)